<a href="https://colab.research.google.com/github/kuir-juach/Career_Guidance_chatbot/blob/main/Education_chapbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv("/content/Career_Guidance_dataset (2).csv")
df = df.dropna()
df = df[['User Query', 'Career Description']]

In [3]:
# Load dataset
df = pd.read_csv("/content/Career_Guidance_dataset (2).csv")
df = df.dropna()
df = df[['User Query', 'Career Description']]

In [4]:
# Add a prefix for T5 task
df['User Query'] = "chatbot: " + df['User Query']

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['User Query'].tolist(), df['Career Description'].tolist(), test_size=0.1, random_state=42
)

In [5]:
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize the dataset
def tokenize_function(inputs, targets):
    model_inputs = tokenizer(inputs, max_length=64, padding="max_length", truncation=True, return_tensors="tf")
    labels = tokenizer(targets, max_length=64, padding="max_length", truncation=True, return_tensors="tf")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_encodings = tokenize_function(train_texts, train_labels)
val_encodings = tokenize_function(val_texts, val_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_ids=train_encodings["input_ids"], attention_mask=train_encodings["attention_mask"]),
    train_encodings["labels"]
)).shuffle(100).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_ids=val_encodings["input_ids"], attention_mask=val_encodings["attention_mask"]),
    val_encodings["labels"]
)).batch(8)

In [7]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer)

In [8]:
# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# Save the model
model.save_pretrained("./career_guidance_chatbot_t5")
tokenizer.save_pretrained("./career_guidance_chatbot_t5")

Epoch 1/3
34/34 [==============================] - 257s 6s/step - loss: 1.6982 - val_loss: 0.4449
Epoch 2/3
34/34 [==============================] - 194s 6s/step - loss: 0.3878 - val_loss: 0.1297
Epoch 3/3
34/34 [==============================] - 196s 6s/step - loss: 0.1578 - val_loss: 0.0684


('./career_guidance_chatbot_t5/tokenizer_config.json',
 './career_guidance_chatbot_t5/special_tokens_map.json',
 './career_guidance_chatbot_t5/spiece.model',
 './career_guidance_chatbot_t5/added_tokens.json')